In [2]:
import os
import sys
from datatools.analyzer import *

from datatools.maneger import DataManager
import pandas as pd

import csv
import time

import random
random.seed(0)

from transformer_model import *

/home/yamada/.local/share/virtualenvs/twichAI-ydQv36PI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
out_path = "../corpus/novel_formated/"
corpus_name = "novel_segments2.tsv"

In [4]:
import csv
conv_data = []
with open(out_path+corpus_name, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    conv_data = [row for row in reader]

In [34]:
# 状況，過去発話が追加された学習データを構築
import copy
def make_context_added_Src_Tgt(conv_data):

    context_src_str = []
    tgt_str = []
    prev_utt = []
    current_situation = [""]
    for conv in conv_data:
        # 状況が変化したか
        if current_situation[0] != conv[0]:
            current_situation = conv[:-2]
            # エラー対策
            if current_situation==[]:
                current_situation = [""]
            prev_utt = [conv[-2]]
        
        context_src_str.append([current_situation, copy.deepcopy(prev_utt) ])
        prev_utt.append(conv[-1])
        tgt_str.append(conv[-1])
    
    return context_src_str, tgt_str

In [51]:
lim=1000
src_str, tgt_str = make_context_added_Src_Tgt(conv_data[:lim])

In [54]:
tokenizer_src = mecab_tokenize
tokenizer_tgt = mecab_tokenize

In [63]:
from collections import Counter
from torchtext.vocab import Vocab

def build_vocab(texts, tokenizer):
    
    counter = Counter()
    for text in tqdm(texts):
        counter.update(tokenizer(text))
    return Vocab(counter, specials=['<unk>', '<pad>', '<fos>', '<eos>','<sep>', '<cxt>', '<del>'])

In [64]:
src_trainval_str, src_test_str, tgt_trainval_str, tgt_test_str= train_test_split(src_str, tgt_str, test_size=0.20, random_state=5)
print("len=> src_train_val:{0}, src_test:{1}".format(len(src_trainval_str), len(src_test_str)))

src_train_str, src_val_str, tgt_train_str, tgt_val_str= train_test_split(src_trainval_str, tgt_trainval_str, test_size=0.10, random_state=5)

print("len=> src_train:{0}, src_val:{1}".format(len(src_train_str), len(src_val_str)))

len=> src_train_val:800, src_test:200
len=> src_train:720, src_val:80


In [65]:
src_train_str_set = list(set(sum(sum(src_train_str, []), [])))

In [66]:
vocab_src = build_vocab(src_train_str_set, tokenizer=tokenizer_src)
vocab_tgt = build_vocab(tgt_train_str, tokenizer=tokenizer_tgt)

100%|██████████| 720/720 [00:00<00:00, 21298.99it/s]


In [67]:
PAD_IDX = vocab_src.stoi["<pad>"]
PAD_IDX

1

In [ ]:
filler_func_sep = lambda L: [*L, "<sep>"]
def fill_SYMBOL_SEP(L):
    sep = list(map(filler_func_sep, L[:-1]))
    sep.append(L[-1])
    return sep

In [75]:
print(fill_SYMBOL_SEP(src_train_str[0][0]))
src_train_str[0][0]

[['そ', 'う', '言', 'っ', 'て', '俺', 'は', '頭', 'を', '下', 'げ', 'た', '。', '[SEP]'], ['…', 'そ', 'し', 'て', '長', 'い', '長', 'い', '沈', '黙', 'が', '訪', 'れ', '。', 'み', 'ん', 'な', 'に', '見', '守', 'ら', 'れ', '。', '遂', 'に', '透', '子', 'ち', 'ゃ', 'ん', 'も', '折', 'れ', 'た', 'よ', 'う', 'で', '。', '[SEP]'], '半ギレながらも、俺と大会に出ることを了承してくれたんだ。']


['そう言って俺は頭を下げた。',
 '…そして長い長い沈黙が訪れ。みんなに見守られ。遂に透子ちゃんも折れたようで。',
 '半ギレながらも、俺と大会に出ることを了承してくれたんだ。']

In [ ]:
def data_process(texts_src, texts_tgt, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt):
    
    data = []
    for (src, tgt) in zip(texts_src, texts_tgt):
        src_tensor = torch.tensor(
            convert_text_to_indexes(text=src, vocab=vocab_src, tokenizer=tokenizer_src, mode="src"),
            dtype=torch.long
        )
        tgt_tensor = torch.tensor(
            convert_text_to_indexes(text=tgt, vocab=vocab_tgt, tokenizer=tokenizer_tgt, mode="tgt"),
            dtype=torch.long
        )
        data.append((src_tensor, tgt_tensor))
        
    return data

def convert_text_to_indexes(text, vocab, tokenizer, mode="src"):
    if mode=="src":
        sit = text[0]
        segments = []
        for s in sit:
            segments += [vocab[token] for token in tokenizer(text.strip("\n"))] + [vocab['<sep>']]
        # 最後消す
        segments.pop()
    # 
    elif mode=="tgt":
        return [vocab['<fos>']] + [
            vocab[token] for token in tokenizer(text.strip("\n"))
        ] + [vocab['<eos>']]
    else:
        return []
        